In [1]:
# neural nets are essentially trying to fit a fnx onto 

# 28 x 28 pixel image
# as the images are greyscale, each pixel has value 0 (black) to 255 (white)

# represent this first as a matrix, each row being an example and having 784 columns
# corresponding to each pixel in the image, brightness of pixels

# transpose this matrix such that each column is an example and each column has 784 rows

# input layer: 784 nodes for each pixel
# hiden layer: 10 nodes
# ouput laye: 10 nodes for each possible digit classification

In [2]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
# importing training data as pandas dataframe
curr_dir = os.getcwd()
path = os.path.join(curr_dir, '../train.csv')

data = pd.read_csv(path)

In [4]:
# to do the maths use numpy array
data = np.array(data)
m, n = data.shape # m is the rows, n is columns
np.random.shuffle(data)

In [5]:
# saving some for test, 1000 
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

In [6]:
# rest is training data
data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [7]:
print("Shape of data")
print("X_train: ", X_train.shape)
print("Y_train: ", Y_train.shape)

Shape of data
X_train:  (784, 41000)
Y_train:  (41000,)


In [8]:
def init_params():
    w1 = np.random.randn(10, 784)  # random starting weights from -0.5 to 0.5 of size 10 by 784 since we are 
                                         # multiplying weight by  input which is a column of the data 784 x 1
    b1 = np.random.randn(10, 1)     # random bais weight 10 x 1 because ouput of ^ mult 784x1 * 10x784 is 10x1
    w2 = np.random.randn(10, 10) 
    b2 = np.random.randn(10, 1) 
    return w1, b1, w2, b2

In [9]:
# forward propegation
# running image through the network to see what network outputs
# A0 = x the input   (input layer 784 x m) 
# Z1 = w1 * A0 + b1   (weight (10 x 784) dot input + bias (10 x 1)) => 10 x m output) (think of this as the synapes)
# A1 = ReLU(Z1)   (apply activation function ReLU still 10x1 matrix)
# Z2 = w2 * A1 + b2 (again this is like the connection between neuron/nodes)
# A2 = softmax(Z2) (another activation function, softmax, which helps turn the outputs into a probability)

# goes through each element in Z, returns relu of it
def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    Z_shifted = Z - np.max(Z)
    return np.exp(Z_shifted) / np.sum(np.exp(Z_shifted))  # makes it such that the values of Z will sum to 1, giving percentage

def forward_prop(w1, b1, w2, b2, X):
    Z1 = w1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = w2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

In [10]:
# what are the correct weights and biases? that is determined in
# backwards propegation
# find out how much the prediction is off from the correct result, error, and determine how much the previous weight
# and biases contriubte to this, and then adjust
# dZ2 = A2 - Y    (take prediction and subtract the correct value, note that the node outputs a 10x1 matrix,
#                  the correct matrix is similar execpt every row is 0 except the index for the correct number)
# dw2 = 1/m * dZ2 * A1^T   (derivative of loss fnx with respect to the weights)
# db2 = 1/m * SUM(dZ2)   (average of the errors)
# dZ1 = w2^T * dZ2 * f'()   (this is sortve like propegation in reverse)
# dw1 = 1/m * dZ! * A0^T   (derivative of loss fnx with respect to the weights)
# db1 = 1/m * SUM(dZ1)

def one_hot(Y):
    # create the label matrix for each data and set the correct identifier as 1
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T # remember to flip because we want columns to be data
    return one_hot_Y

def dydxReLU(Z):
    return Z > 0 # if x less than 0 returns 0 if greater return 1, slope of y=z

def back_prop(Z1, A1, Z2, A2, w1, w2, X, Y): # to compare with actual 
    one_hot_Y = one_hot(Y)
    m = one_hot_Y.size
    dZ2 = A2 - one_hot_Y
    dw2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2)
    dZ1 = w2.T.dot(dZ2) * dydxReLU(Z1)
    dw1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1)
    return dw1, db1, dw2, db2

In [11]:
# update parameters usinig learning rate r (hyper parameter)
# w1 = w1 - r * dw1
# b1 = b1 - r * db1
# w2 = w2 - r * dw2
# b2 = b2 - r * db2

def update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, rate):
    w1 = w1 - rate * dw1
    b1 = b1 - rate * db1
    w2 = w2 - rate * dw2
    b2 = b2 - rate * db2
    return w1, b1, w2, b2

In [12]:
# determine accuracy of model
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

In [13]:
# describe gradient descent
# iterations how many times updating params

def gradient_descent(X, Y, iterations, rate):
    w1, b1, w2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(w1, b1, w2, b2, X)
        dw1, db1, dw2, db2 = back_prop(Z1, A1, Z2, A2, w1, w2, X, Y)
        w1, b1, w2, b2 = update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, rate)
        if i % 50 == 0:
            print("Iteration: ", i)
            print("Accuracy: ", get_accuracy(get_predictions(A2), Y))
    return w1, b1, w2, b2

In [14]:
w1, b1, w2, b2 = gradient_descent(X_train, Y_train, 300, 0.1)

Iteration:  0
Accuracy:  0.15490243902439024
Iteration:  50
Accuracy:  0.19975609756097562
Iteration:  100
Accuracy:  0.13046341463414635
Iteration:  150
Accuracy:  0.09853658536585366
Iteration:  200
Accuracy:  0.09853658536585366
Iteration:  250
Accuracy:  0.09853658536585366
